In [ ]:
green_areas = ''

gid = 'ffif'

buffer_dist = 250

urban_areas = ''

outfeat = ''

In [ ]:
import os

from glass.pys.tm import now_as_str
from glass.wenv.grs import run_grass
from glass.pys.oss import fprop
from glass.gp.prox.bfing import cheese_buffer

In [ ]:
ws = os.path.dirname(outfeat)

loc = f'loc_{now_as_str()}'

# Create GRASS GIS Session
gb = run_grass(ws, location=loc, srs=green_areas)

import grass.script.setup as gs

gs.init(gb, ws, loc, 'PERMANENT')

In [ ]:
# Get Cheese buffer

cbf = cheese_buffer(
    green_areas, buffer_dist, 45,
    os.path.join(ws, loc, f'cb_{fprop(green_areas, "fn")}.shp'),
    uniqueid='jpk', areaf="area_slice",
    epsg=epsg, shpid=gid
)

In [ ]:
from glass.it.shp import shp_to_grs, grs_to_shp
from glass.gp.ovl import grsintersection
from glass.gp.gen import dissolve

In [ ]:
# Import data into GRASS GIS
cbfgrs = shp_to_grs(cbf, asCMD=True)

uareas = shp_to_grs(urban_areas)

In [ ]:
# Intersect cheese buffer with urban regions

igrs = grsintersection(cbfgrs, uareas, 'green_urban', cmd=True)

# Export data
ishp = grs_to_shp(igrs, os.path.join(ws, loc, f'{igrs}.shp'), 'area')

In [ ]:
# Dissolve by ID and direction
idiss = dissolve(ishp, 'green_urban_diss', 'jpk', api='grass')

# Obtain area value
geomattr_to_db(idiss, "garea", "area", "boundary", unit='meters')